In [1]:
import os
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import csv

load_pickle() => is used to read the CIFAR 10 dataset and stores in binary format 

Data files are loaded in 5 batches, then the images and label's were extracted and were transformed from (C,H,W) to (H,W,C) using transpose(1,2,0)

Data augmentation and normalization was done using tensor, mean and standard deviation to stablize and speedup training. 

Images were randomly cropeed to 32x32 pixels and padding of 4 pixels was added to help in training so that images with slight shifts were also classified correctly 

DataLoader is used to help the model in learning data in random instead of leaning in a particular order every epoch, 4 worker threads were used to speedup the training process by parallelizing it. 

Batch size of 256 is used to process many samples of data simultaneously 

In [2]:
import pickle
import numpy as np

import pickle
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

def load_pickle(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
    return data

batch_files = [
    "./data_batch_1",
    "./data_batch_2",
    "./data_batch_3",
    "./data_batch_4",
    "./data_batch_5"
]

train_data, train_labels = [], []
for file in batch_files:
    batch = load_pickle(file)
    train_data.append(batch[b'data'])       
    train_labels.extend(batch[b'labels'])

train_data = np.vstack(train_data).reshape(-1, 3, 32, 32)
train_labels = np.array(train_labels)

class CIFAR10Dataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx].transpose(1, 2, 0)  
        img = Image.fromarray(img)
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.2010])
])

trainset = CIFAR10Dataset(train_data, train_labels, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=512, shuffle=True, num_workers=4)

mish activation function is used to provide smooth gradients and improving the convergence of the network
Mish(x) = x . tanh(ln(1+e^x))

SE block generates attention weights which scales the output of the convolutional layers

Basic block has 2 convolutional layers with batch normalization and mish activation, skip connection is added to allow the flow of gradients solving the vanishing gradient problem


Modified ResNet18 - there are 4 major layers to capture different levels of abstraction that is built using stacking the basic blocks

Adaptive Average Pooling layers compresses the spatial dimensions into 1x1 per channel - used to summerize each feature map into a single value per channel

Normalization is added here as well for stability and speeding up the training 


While training the model all the transformations and conversions that were used while reading and procressing was added here.

Using cross entropy loss to evaluate how well it will align with the prediction 

SGD is used for stable learning over time 

Weight decay is used to reguarlize the model, tried with 5e-3, 1e-3

CosineAnnealingWarmRestarts is used to dynamically change learning rate based on the loss and number of epochs 

Training Loss

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from tqdm import tqdm
from torchsummary import summary

class Mish(nn.Module):
    def forward(self, x):
        return x * torch.tanh(F.softplus(x))

class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, 1),
            Mish(),
            nn.Conv2d(channels // reduction, channels, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return x * self.se(x)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.se = SEBlock(planes)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = Mish()(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.se(out)
        out += self.shortcut(x)
        return Mish()(out)

class ResNet18(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, dropout_prob=0.5):
        super(ResNet18, self).__init__()
        self.in_planes = 32

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)

        self.layer1 = self._make_layer(block, 32, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 64, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 256, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(dropout_prob)  
        self.fc = nn.Linear(256, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = Mish()(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = self.dropout(out) 
        out = torch.flatten(out, 1)
        return self.fc(out)

def train():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.AutoAugment(),  
        transforms.ToTensor(),         
        transforms.RandomErasing(),     
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
    
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
    
    trainset = CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    trainloader = DataLoader(trainset, batch_size=256, shuffle=True, num_workers=4)
    
    testset = CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    testloader = DataLoader(testset, batch_size=256, shuffle=False, num_workers=4)
    
    model = ResNet18(BasicBlock, [2, 2, 2, 2]).to(device)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=1e-3)
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
    
    num_epochs = 100
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        total_train = 0
        
        for inputs, labels in tqdm(trainloader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total_train += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()
        
        avg_train_loss = train_loss / total_train
        train_acc = 100. * train_correct / total_train
        
        model.eval()
        test_loss = 0.0
        test_correct = 0
        total_test = 0
        
        with torch.no_grad():
            for inputs, labels in tqdm(testloader, desc=f"Testing Epoch {epoch+1}/{num_epochs}"):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                total_test += labels.size(0)
                test_correct += predicted.eq(labels).sum().item()
        
        avg_test_loss = test_loss / total_test
        test_acc = 100. * test_correct / total_test
        
        scheduler.step()
        
        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"    Training Loss: {avg_train_loss:.4f} | Training Acc: {train_acc:.2f}%")
        print(f"    Testing Loss: {avg_test_loss:.4f}  | Testing Acc: {test_acc:.2f}%")
        torch.save(model.state_dict(), "1optimized_resnet18.pth")
    
    print("\n Model Summary After Training:")
    summary(model, (3, 32, 32))
    
if __name__ == '__main__':
    train()


Testing Epoch 1/100: 100%|██████████| 40/40 [00:00<00:00, 49.60it/s]


Epoch 1/100:
    Training Loss: 2.0521 | Training Acc: 27.17%
    Testing Loss: 1.6034  | Testing Acc: 48.82%


Testing Epoch 2/100: 100%|██████████| 40/40 [00:00<00:00, 55.84it/s]


Epoch 2/100:
    Training Loss: 1.7645 | Training Acc: 41.86%
    Testing Loss: 1.4413  | Testing Acc: 56.87%


Testing Epoch 3/100: 100%|██████████| 40/40 [00:00<00:00, 55.28it/s]


...

Testing Epoch 83/100: 100%|██████████| 40/40 [00:00<00:00, 56.59it/s]


Epoch 83/100:
    Training Loss: 1.1103 | Training Acc: 73.60%
    Testing Loss: 0.9221  | Testing Acc: 82.23%


Testing Epoch 84/100: 100%|██████████| 40/40 [00:00<00:00, 56.31it/s]


Epoch 84/100:
    Training Loss: 1.1136 | Training Acc: 73.55%
    Testing Loss: 0.8643  | Testing Acc: 84.82%


Training Epoch 85/100:  67%|██████▋   | 131/196 [00:08<00:04, 14.61it/s]

Code to Create the submissions.csv file after unpickling the nolabel data and running an inference on it

In [4]:
import torch
import torchvision.transforms as transforms
import pandas as pd
import pickle

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = ResNet18(BasicBlock, [2, 2, 2, 2]).to(device)
state_dict = torch.load("optimized_resnet18.pth", map_location=device)
model.load_state_dict(state_dict)
model.to(device)
model.eval()  
print("Model loaded successfully!")

def load_cifar_batch(file):
    with open(file, 'rb') as fo:
        batch = pickle.load(fo, encoding='bytes')
    return batch

file_path = "./cifar_test_nolabel.pkl"
cifar10_batch = load_cifar_batch(file_path)

test_images = cifar10_batch[b'data']  
image_ids = cifar10_batch[b'ids'] 

print(f"Test set loaded: {test_images.shape}")

test_images = torch.tensor(test_images, dtype=torch.float32).permute(0, 3, 1, 2) / 255.0
transform = transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
test_images = torch.stack([transform(img) for img in test_images])


test_images = test_images.to(device)  
batch_size = 64  

predictions = []
with torch.no_grad():
    for i in range(0, len(test_images), batch_size):
        batch = test_images[i:i + batch_size]  
        outputs = model(batch)  
        _, predicted_labels = torch.max(outputs, 1) 
        predictions.extend(predicted_labels.cpu().numpy())  

print("Inference completed!")

submission_df = pd.DataFrame({
    "ID": image_ids,
    "Labels": predictions
})
submission_df.to_csv("submission.csv", index=False)
print("submission.csv generated.")

Model loaded successfully!
Test set loaded: (10000, 32, 32, 3)
Inference completed!
submission.csv generated.


In [5]:
import pandas as pd

df = pd.read_csv("./submission.csv")
print(df.head())

   ID  Labels
0   0       6
1   1       1
2   2       8
3   3       6
4   4       9


Code to check the distribution of labels for a total of 10,000 images

In [6]:
unique_labels = df['Labels'].nunique()
print(f"Total unique labels: {unique_labels}")

label_counts = df['Labels'].value_counts()
print(label_counts)

Total unique labels: 10
Labels
3    1207
2    1112
8    1079
1    1060
5    1004
7     989
4     943
6     904
9     883
0     819
Name: count, dtype: int64


Code to check the accuracy of the model trained using the test_batch

In [7]:
import torch
import torchvision.transforms as transforms
import pickle

def load_cifar_batch(file):
    with open(file, 'rb') as fo:
        batch = pickle.load(fo, encoding='bytes')
    return batch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = ResNet18(BasicBlock, [2, 2, 2, 2]).to(device)
state_dict = torch.load("optimized_resnet18.pth", map_location=device)
model.load_state_dict(state_dict)
model.to(device)
model.eval()  
print("Model loaded successfully!")

file_path = "test_batch"  
cifar_batch = load_cifar_batch(file_path)

test_images = cifar_batch[b'data']   
true_labels = cifar_batch[b'labels']  

print(f"Test set loaded: {test_images.shape}")

if test_images.shape[1] == 3072:
    test_images = test_images.reshape(-1, 3, 32, 32)
elif test_images.shape[1] == 32:
    test_images = torch.tensor(test_images, dtype=torch.float32).permute(0, 3, 1, 2) / 255.0
else:
    test_images = torch.tensor(test_images, dtype=torch.float32) / 255.0

if not isinstance(test_images, torch.Tensor):
    test_images = torch.tensor(test_images, dtype=torch.float32) / 255.0

if test_images.shape[1] != 3:
    test_images = test_images.permute(0, 3, 1, 2)

normalize_transform = transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                             std=[0.2470, 0.2435, 0.2616])

test_images = torch.stack([normalize_transform(img) for img in test_images])
test_images = test_images.to(device)

true_labels = torch.tensor(true_labels, dtype=torch.long).to(device)

batch_size = 64
total = test_images.size(0)
correct = 0

with torch.no_grad():
    for i in range(0, total, batch_size):
        batch = test_images[i:i+batch_size]
        outputs = model(batch)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == true_labels[i:i+batch_size]).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on test set: {accuracy:.2f}%")

Model loaded successfully!
Test set loaded: (10000, 3072)
Accuracy on test set: 92.72%


Code to Count the number of parameters = 4.5 million

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from tqdm import tqdm
from torchsummary import summary

class Mish(nn.Module):
    def forward(self, x):
        return x * torch.tanh(F.softplus(x))

class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, 1),
            Mish(),
            nn.Conv2d(channels // reduction, channels, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return x * self.se(x)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.se = SEBlock(planes)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = Mish()(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.se(out)
        out += self.shortcut(x)
        return Mish()(out)

class ResNet18(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, dropout_prob=0.5):
        super(ResNet18, self).__init__()
        self.in_planes = 32

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)

        self.layer1 = self._make_layer(block, 32, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 64, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 256, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(dropout_prob)  
        self.fc = nn.Linear(256, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = Mish()(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = F.dropout(out, p=0.5, training=self.training) 
        return self.fc(out)

def continue_training():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
   
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.AutoAugment(),
        transforms.ToTensor(),          
        transforms.RandomErasing(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
    
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
    
    trainset = CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    trainloader = DataLoader(trainset, batch_size=256, shuffle=True, num_workers=4)
    
    testset = CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    testloader = DataLoader(testset, batch_size=256, shuffle=False, num_workers=4)
    
    model = ResNet18(BasicBlock, [2, 2, 2, 2]).to(device)
    model.load_state_dict(torch.load("optimized_resnet18.pth"))

    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=1e-4)
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
    
    num_epochs = 200
    for epoch in range(1):  
        model.train()
        train_loss = 0.0
        train_correct = 0
        total_train = 0
        
        for inputs, labels in tqdm(trainloader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total_train += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()
        
        avg_train_loss = train_loss / total_train
        train_acc = 100. * train_correct / total_train
        
        model.eval()
        test_loss = 0.0
        test_correct = 0
        total_test = 0
        
        with torch.no_grad():
            for inputs, labels in tqdm(testloader, desc=f"Testing Epoch {epoch+1}/{num_epochs}"):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                total_test += labels.size(0)
                test_correct += predicted.eq(labels).sum().item()
        
        avg_test_loss = test_loss / total_test
        test_acc = 100. * test_correct / total_test
        
        scheduler.step()
        
        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"    Training Loss: {avg_train_loss:.4f} | Training Acc: {train_acc:.2f}%")
        print(f"    Testing Loss: {avg_test_loss:.4f}  | Testing Acc: {test_acc:.2f}%")
        
        torch.save(model.state_dict(), "optimized_resnet18.pth")
    
    print("\n Model Summary After Training:")
    summary(model, (3, 32, 32))
    
if __name__ == '__main__':
    continue_training()

Testing Epoch 1/200: 100%|██████████| 40/40 [00:00<00:00, 49.44it/s]


Epoch 1/200:
    Training Loss: 0.7354 | Training Acc: 90.15%
    Testing Loss: 0.6711  | Testing Acc: 92.73%

 Model Summary After Training:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
       BatchNorm2d-2           [-1, 32, 32, 32]              64
            Conv2d-3           [-1, 32, 32, 32]           9,216
       BatchNorm2d-4           [-1, 32, 32, 32]              64
            Conv2d-5           [-1, 32, 32, 32]           9,216
       BatchNorm2d-6           [-1, 32, 32, 32]              64
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 2, 1, 1]              66
              Mish-9              [-1, 2, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]              96
          Sigmoid-11             [-1, 32, 1, 1]               0
          SEBlock-12     

This block of code was used to continue training the model from where it had left off due to HPC crashing. 

In our code, we are uploading the model after every epoch. When HPC crashes and we have ro restart out training, then we continue training from the same number of epoch using the latest model. This way we could train the model for 200 epochs over multiple HPC sessions even if HPC crashed. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from tqdm import tqdm
from torchsummary import summary

class Mish(nn.Module):
    def forward(self, x):
        return x * torch.tanh(F.softplus(x))

class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, 1),
            Mish(),
            nn.Conv2d(channels // reduction, channels, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return x * self.se(x)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.se = SEBlock(planes)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = Mish()(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.se(out)
        out += self.shortcut(x)
        return Mish()(out)

class ResNet18(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, dropout_prob=0.5):
        super(ResNet18, self).__init__()
        self.in_planes = 32

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)

        self.layer1 = self._make_layer(block, 32, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 64, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 256, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(dropout_prob)  
        self.fc = nn.Linear(256, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = Mish()(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = F.dropout(out, p=0.5, training=self.training) 
        return self.fc(out)

def continue_training():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
   
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.AutoAugment(),
        transforms.ToTensor(),          
        transforms.RandomErasing(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
    
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
    
    trainset = CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    trainloader = DataLoader(trainset, batch_size=256, shuffle=True, num_workers=4)
    
    testset = CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    testloader = DataLoader(testset, batch_size=256, shuffle=False, num_workers=4)
    
    model = ResNet18(BasicBlock, [2, 2, 2, 2]).to(device)
    model.load_state_dict(torch.load("optimized_resnet18.pth"))

    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=1e-4)
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
    
    num_epochs = 200
    for epoch in range(84,200):  
        model.train()
        train_loss = 0.0
        train_correct = 0
        total_train = 0
        
        for inputs, labels in tqdm(trainloader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total_train += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()
        
        avg_train_loss = train_loss / total_train
        train_acc = 100. * train_correct / total_train
        
        model.eval()
        test_loss = 0.0
        test_correct = 0
        total_test = 0
        
        with torch.no_grad():
            for inputs, labels in tqdm(testloader, desc=f"Testing Epoch {epoch+1}/{num_epochs}"):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                total_test += labels.size(0)
                test_correct += predicted.eq(labels).sum().item()
        
        avg_test_loss = test_loss / total_test
        test_acc = 100. * test_correct / total_test
        
        scheduler.step()
        
        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"    Training Loss: {avg_train_loss:.4f} | Training Acc: {train_acc:.2f}%")
        print(f"    Testing Loss: {avg_test_loss:.4f}  | Testing Acc: {test_acc:.2f}%")
        
        torch.save(model.state_dict(), "optimized_resnet18.pth")
    
    print("\n Model Summary After Training:")
    summary(model, (3, 32, 32))
    
if __name__ == '__main__':
    continue_training()

Testing Epoch 85/200: 100%|██████████| 40/40 [00:00<00:00, 47.41it/s]


Epoch 85/200:
    Training Loss: 1.2218 | Training Acc: 73.90%
    Testing Loss: 0.8299  | Testing Acc: 86.35%


Testing Epoch 86/200: 100%|██████████| 40/40 [00:00<00:00, 58.09it/s]


Epoch 86/200:
    Training Loss: 1.0564 | Training Acc: 79.53%
    Testing Loss: 0.8231  | Testing Acc: 86.20%


Testing Epoch 87/200: 100%|██████████| 40/40 [00:00<00:00, 54.99it/s]


...

Testing Epoch 191/200: 100%|██████████| 40/40 [00:00<00:00, 57.18it/s]


Epoch 191/200:
    Training Loss: 0.7103 | Training Acc: 91.28%
    Testing Loss: 0.6445  | Testing Acc: 93.98%


Testing Epoch 192/200: 100%|██████████| 40/40 [00:00<00:00, 56.75it/s]


Epoch 192/200:
    Training Loss: 0.7080 | Training Acc: 91.37%
    Testing Loss: 0.6454  | Testing Acc: 93.90%


Training Epoch 193/200:  21%|██        | 41/196 [00:03<00:12, 12.79it/s]